# Conversational UI Chatbot App with Gemini, LangChain and Streamlit

Here we will build a advanced Conversational UI-based chatbot using LangChain and Streamlit with the following features:

- Custom Landing Page
- Conversational memory

## Install App and LLM dependencies

In [1]:
!pip install langchain==0.1.12 -q
!pip install langchain-google-genai==0.0.7 -q
!pip install langchain-community==0.0.29 -q
!pip install streamlit==1.32.2 -q
!pip install pyngrok==7.1.5 -q
!pip install google-generativeai>=0.3.2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 908.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 824.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
google-cloud-bigquery 3.31.0 requires packaging>=24.2.0, but you have packaging 23.2 which is inc

## Load Gemini API Credentials

Here we load it from a file so we don't explore the credentials on the internet by mistake

In [3]:
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

## Basic Streamlit UI without Memory

This simple version shows how to:

1. Create a basic Streamlit interface
2. Connect directly to Gemini
3. Process basic Q&A without memory

In [4]:
%%writefile basic_app.py

import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI

st.title("Basic AI Assistant (No Memory)")

# Initialize the LLM
gemini = ChatGoogleGenerativeAI(model='gemini-2.0-flash-001',
                               temperature=0.1)

# Simple input/output
if user_input := st.chat_input("Ask a question"):
    st.chat_message("human").write(user_input)

    with st.chat_message("ai"):
        response = gemini.invoke(user_input)
        st.write(response.content)

Writing basic_app.py


## Start the app

In [5]:
!streamlit run basic_app.py --server.port=8989 &>./logs.txt &

In [6]:
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

ngrok.set_auth_token(userdata.get('NGROK_API_KEY'))

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://378c-34-138-160-11.ngrok-free.app


## Remove running app processes

In [7]:
ngrok.kill()

In [8]:
!ps -ef | grep streamlit

root        2463       1  1 09:19 ?        00:00:04 /usr/bin/python3 /usr/local/bin/streamlit run basic_app.py --server.port=8989
root        3971    1533  0 09:25 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root        3973    3971  0 09:25 ?        00:00:00 grep streamlit


In [10]:
!sudo kill -9 2463

## Part 2: Showing Messages with Manual Memory

This demonstrates:

1. How to manually implement memory in Streamlit
2. The challenge of formatting context for the LLM
3. Why specialized tools like LangChain help

In [11]:
%%writefile manual_app.py

import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI

st.title("AI Assistant with Manual Memory")

# Initialize the LLM
gemini = ChatGoogleGenerativeAI(model='gemini-2.0-flash-001',
                               temperature=0.1)

# Initialize session state for memory
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display existing messages
for message in st.session_state.messages:
    st.chat_message(message["role"]).write(message["content"])

# Process new input
if user_input := st.chat_input("Ask a question"):
    # Add user message to history
    st.session_state.messages.append({"role": "human", "content": user_input})
    st.chat_message("human").write(user_input)

    # Manually create a context string from history
    context = "Previous conversation:\n"
    for msg in st.session_state.messages:
        context += f"{msg['role']}: {msg['content']}\n"

    with st.chat_message("ai"):
        # Send context + new question
        full_prompt = context + "\nPlease respond to the last question."
        response = gemini.invoke(full_prompt)
        st.write(response.content)

        # Add AI response to history
        st.session_state.messages.append({"role": "ai", "content": response.content})

Writing manual_app.py


In [12]:
!streamlit run manual_app.py --server.port=8989 &>./logs.txt &

In [13]:
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

ngrok.set_auth_token(userdata.get('NGROK_API_KEY'))

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://2582-34-138-160-11.ngrok-free.app


In [14]:
ngrok.kill()

In [15]:
!ps -ef | grep streamlit

root        4795       1  0 09:28 ?        00:00:03 /usr/bin/python3 /usr/local/bin/streamlit run manual_app.py --server.port=8989
root        6852    1533  0 09:37 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root        6854    6852  0 09:37 ?        00:00:00 grep streamlit


In [16]:
!sudo kill -9 4795

## Part 3: Full LangChain Integration

In [17]:
%%writefile langchain_app.py

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from operator import itemgetter
import streamlit as st

# Customize initial app landing page
st.set_page_config(page_title="AI Assistant", page_icon="🤖")
st.title("Welcome I am AI Assistant 🤖")

# Load a connection to Gemini LLM
gemini = ChatGoogleGenerativeAI(model='gemini-2.0-flash-001',
                               temperature=0.1,
                               convert_system_message_to_human=True)

# Add a basic system prompt for LLM behavior
SYS_PROMPT = """
              Act as a helpful assistant and answer questions to the best of your ability.
              Do not make up answers.
              """

# Create a prompt template for langchain to use history to answer user prompts
prompt = ChatPromptTemplate.from_messages(
  [
    ("system", SYS_PROMPT),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
  ]
)

# Create a basic llm chain
llm_chain = (
  prompt
  |
  gemini
)

# Store conversation history in Streamlit session state
streamlit_msg_history = StreamlitChatMessageHistory()

# Create a conversation chain
conversation_chain = RunnableWithMessageHistory(
  llm_chain,
  lambda session_id: streamlit_msg_history,  # Accesses memory
  input_messages_key="input",
  history_messages_key="history",
)

# Render current messages from StreamlitChatMessageHistory
for msg in streamlit_msg_history.messages:
  st.chat_message(msg.type).write(msg.content)

# If user inputs a new prompt, display it and show the response
if user_prompt := st.chat_input():
  st.chat_message("human").write(user_prompt)
  # This is where response from the LLM is shown
  with st.chat_message("ai"):
    config = {"configurable": {"session_id": "any"}}
    # Get llm response
    response = conversation_chain.invoke({"input": user_prompt}, config)
    st.markdown(response.content) # Display response directly

Writing langchain_app.py


## Start the app

In [18]:
!streamlit run langchain_app.py --server.port=8989 &>./logs.txt &

In [19]:
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

ngrok.set_auth_token(userdata.get('NGROK_API_KEY'))

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://bc26-34-138-160-11.ngrok-free.app


## Remove running app processes

In [ ]:
ngrok.kill()

In [ ]:
!ps -ef | grep streamlit

root       54456       1  7 14:25 ?        00:00:03 /usr/bin/python3 /usr/local/bin/streamlit run langchain_app.py --server.port=8989
root       54654   13964  0 14:26 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root       54656   54654  0 14:26 ?        00:00:00 grep streamlit


In [ ]:
!sudo kill -9 54456